# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Data Prepare

In [5]:
# 去除空行与浮点化操作
data = pd.read_csv('../Data/aiops_data.csv')
_data = data.copy()
_data = _data.dropna()
_data = _data.astype('float')

In [214]:
# 划分出90个样本作为测试集
# 划分出2360个样本作为训练集
sample_num = 30
test_index = []

import random
random.seed( 10 )

for a in range(3): 
    b = np.array( _data[_data[str(a+1)]==1].index.tolist() )
    sample_list = [i for i in range(len(b))]
    sample_list = random.sample(sample_list, sample_num)
    test_index.append(b[sample_list])
c = np.concatenate(test_index)
test = _data.loc[c,:]
train = _data.drop(index=c)

In [221]:
label = test[['1','2','3','4','5','6']].values

# likelihood

In [198]:
def NormalPara(X): # NomalPara接受一组array
    return np.mean(X,axis=0), np.cov(X.T)

# 取样：root=1和root=0
R1_1 = train[train['1']==1]
R1_0 = train[train['1']==0]
R2_1 = train[train['2']==1]
R2_0 = train[train['2']==0]
R3_1 = train[train['3']==1]
R3_0 = train[train['3']==0]
# 取出有效的特征
R1_1 = R1_1[['feature15','feature13']]
R1_0 = R1_0[['feature15','feature13']]
R2_1 = R2_1[[\
    "feature19",\
    "feature61_0", "feature61_1","feature61_2",  "feature61_3","feature61_4","feature61_5","feature61_6","feature61_7",\
    "feature69_0", "feature69_1","feature69_2","feature69_3","feature69_4",  "feature69_5","feature69_6", \
    "feature28_0", "feature28_1","feature28_2",  "feature28_3","feature28_4","feature28_5","feature28_6","feature28_7",\
    "feature36_0", "feature36_1","feature36_2",  "feature36_3","feature36_4", "feature36_5","feature36_6","feature36_7"\
    ]]
R2_0 = R2_0[[\
    "feature19",\
    "feature61_0", "feature61_1","feature61_2",  "feature61_3","feature61_4","feature61_5","feature61_6","feature61_7",\
    "feature69_0", "feature69_1","feature69_2","feature69_3","feature69_4",  "feature69_5","feature69_6", \
    "feature28_0", "feature28_1","feature28_2",  "feature28_3","feature28_4","feature28_5","feature28_6","feature28_7",\
    "feature36_0", "feature36_1","feature36_2",  "feature36_3","feature36_4", "feature36_5","feature36_6","feature36_7"\
    ]]
R3_1 = R3_1[[\
    "feature60",\
    "feature61_0", "feature61_1","feature61_2",  "feature61_3","feature61_4","feature61_5","feature61_6","feature61_7",\
    "feature69_0", "feature69_1","feature69_2","feature69_3","feature69_4",  "feature69_5","feature69_6", \
    "feature20_0","feature20_1","feature20_2","feature20_3","feature20_4", "feature20_5","feature20_6","feature20_7", 
    "feature28_0","feature28_1","feature28_2","feature28_3","feature28_4","feature28_5","feature28_6","feature28_7",\
    "feature36_0", "feature36_1","feature36_2","feature36_3","feature36_4","feature36_5","feature36_6","feature36_7"\
    ]]
R3_0 = R3_0[[\
    "feature60",\
    "feature61_0", "feature61_1","feature61_2",  "feature61_3","feature61_4","feature61_5","feature61_6","feature61_7",\
    "feature69_0", "feature69_1","feature69_2","feature69_3","feature69_4",  "feature69_5","feature69_6", \
    "feature20_0","feature20_1","feature20_2","feature20_3","feature20_4", "feature20_5","feature20_6","feature20_7", 
    "feature28_0","feature28_1","feature28_2","feature28_3","feature28_4","feature28_5","feature28_6","feature28_7",\
    "feature36_0", "feature36_1","feature36_2","feature36_3","feature36_4","feature36_5","feature36_6","feature36_7"\
    ]]
# 按列计算均值；计算属性之间的协方差
mu_R1_0,cov_R1_0 = NormalPara(R1_0.values)
mu_R1_1,cov_R1_1 = NormalPara(R1_1.values)

from scipy.stats import multivariate_normal
R1_pdf.append(multivariate_normal(mu_R1_0,cov_R1_0))
R1_pdf.append(multivariate_normal(mu_R1_1,cov_R1_1))
R1_pdf,R2_pdf,R3_pdf=[],[],[]
R2_pdf.append(multivariate_normal(mu_R2_0,cov_R2_0))
R2_pdf.append(multivariate_normal(mu_R2_1,cov_R2_1+np.eye(32)*0.0000001))# 奇异矩阵添加约束
R3_pdf.append(multivariate_normal(mu_R3_0,cov_R3_0))
R3_pdf.append(multivariate_normal(mu_R3_1,cov_R3_1))

 # prior
pri_R1, pri_R2, pri_R3 = [0.5,0.5],[0.5,0.5],[0.5,0.5]

# pro_product()
def pro_product(sample,r1,r2,r3):
    likelihood1 = R1_pdf[r1].pdf(sample[['feature15','feature13']].values)
    likelihood2 = R2_pdf[r2].pdf(sample[[\
    "feature19",\
    "feature61_0", "feature61_1","feature61_2",  "feature61_3","feature61_4","feature61_5","feature61_6","feature61_7",\
    "feature69_0", "feature69_1","feature69_2","feature69_3","feature69_4",  "feature69_5","feature69_6", \
    "feature28_0", "feature28_1","feature28_2",  "feature28_3","feature28_4","feature28_5","feature28_6","feature28_7",\
    "feature36_0", "feature36_1","feature36_2",  "feature36_3","feature36_4", "feature36_5","feature36_6","feature36_7"\
    ]].values)
    likelihood3 = R3_pdf[r3].pdf(sample[[\
    "feature60",\
    "feature61_0", "feature61_1","feature61_2",  "feature61_3","feature61_4","feature61_5","feature61_6","feature61_7",\
    "feature69_0", "feature69_1","feature69_2","feature69_3","feature69_4",  "feature69_5","feature69_6", \
    "feature20_0","feature20_1","feature20_2","feature20_3","feature20_4", "feature20_5","feature20_6","feature20_7", 
    "feature28_0","feature28_1","feature28_2","feature28_3","feature28_4","feature28_5","feature28_6","feature28_7",\
    "feature36_0", "feature36_1","feature36_2","feature36_3","feature36_4","feature36_5","feature36_6","feature36_7"\
    ]].values)
    return likelihood1*likelihood2*likelihood3*pri_R1[r1]*pri_R1[r2]*pri_R1[r3]

# Test

In [209]:
def predict(sample):
    record=[] 
    for r1 in range(2): 
        for r2 in range(2):
            for r3 in range(2):
                record.append([r1,r2,r3,pro_product(sample,r1,r2,r3)]) 
    record = np.array(record)
    record[:,3] = record[:,3]/sum(record[:,3]) 
    return sum(record[record[:,0]==1][:,3]),sum(record[record[:,1]==1][:,3]),sum(record[record[:,2]==1][:,3])

In [223]:
pred = np.zeros([test.shape[0],6])
for i in range(test.shape[0]):
    pred[i,0], pred[i,1], pred[i,2] = predict(test[i:i+1])
# 设立阈值
pred = pred > 0.8

In [224]:
plus=np.sum(pred*label,axis=1)
minus=np.sum(pred*(1-label),axis=1)
np.mean((plus-minus)/np.sum(label,axis=1))

0.4222222222222222